In [ ]:
#!pip install sdv

In [5]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from joypy import joyplot
sns.set(style="darkgrid")
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
from sdv.tabular import GaussianCopula
from sdv.lite import TabularPreset
from sdv.tabular import CTGAN
from sdv.tabular import CopulaGAN
from sdv.tabular import TVAE
from sdv.evaluation import evaluate
import torch
import warnings
warnings.filterwarnings("ignore")
from synthetic_data import syntheticData, inspectData

### Import dataset

In [6]:
real_data = pd.read_csv('OPUS.csv')
for col in real_data.columns:
      real_data[col] = pd.to_numeric(real_data[col],errors = 'coerce')


In [7]:
def missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    
    # Drop columns with >= 25 % of missing values 
    to_keep = (missing_value_df[missing_value_df.iloc[:,-1]<=25.0].index)
    df_new = df.drop(df.columns.difference(to_keep), axis=1)

    return [missing_value_df, df_new]

def duplicated (df):
    df_new = df.loc[:,~df.columns.duplicated()]
    return df_new
missing = missing_values(real_data)

In [8]:
real_data = real_data.copy()

### Synthetic data class

In [9]:
### Executing syntheticData class
synthetic_data = syntheticData(real_data,1000)
synthetic_data.train_models()

### Evaluation 

In [10]:
synthetic_data.evaluate()
print('ks metrics of all model')
synthetic_data.ks_metrics

ks metrics of all model


{'gaussian': array([0.89180626]),
 'tabular': array([0.67324997]),
 'ctgan': array([0.85559943]),
 'copula_gan': array([0.86716581]),
 'tvae': array([0.83113258])}

In [9]:
print('Best model ks metrics')
synthetic_data.selected_model

Best model ks metrics


{'best_model': 'gaussian', 'ks_score': array([0.89495494])}

In [10]:
synthetic_data = synthetic_data.sample_data

### Inspect data class

In [11]:
### executing inspect data class
inspect_data = inspectData(real_data,synthetic_data)

In [18]:
inspect_data.density_histogram()

In [19]:
inspect_data.kernel_density()

In [20]:
inspect_data.ridgeline_plot()

In [23]:
inspect_data.group_violin_plots()

In [ ]:
# import numpy as np # Data manipulation  
# import matplotlib.pyplot as plt 
# import seaborn 
# c = real_data.corr() 
# plt.figure(figsize=(20,20)) 
# seaborn.heatmap(c, cmap='RdYlGn_r',vmin=-1, vmax=1) 
# plt.show()

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

corr1 = real_data.corr() #your dataframe

# figsize=(6, 6) control width and height
# dpi = 600, I 
plt.figure(figsize=(20, 20), 
           dpi = 600) 
 
# parameter annot_kws={"size": 8} control corr values font size
sns.heatmap(corr1, cmap="Blues",vmin=-1, vmax=1)

plt.tick_params(axis = 'x', labelsize = 12) # x font label size
plt.tick_params(axis = 'y', labelsize = 12) # y font label size

In [62]:
# c = synthetic_data.corr() 
# plt.figure(figsize=(18,18)) 
# seaborn.heatmap(c, cmap='RdYlGn_r',vmin=-1, vmax=1) 
# plt.show()
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt

In [26]:
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt



n3 = corr1.drop('death',axis=1)
n4 = n3.drop('on which day the death occured',axis=1)
n5 = n4.drop('death',axis=0)
n6 = n5.drop('on which day the death occured',axis=0)


In [27]:
m3 = corr2.drop('death',axis=1)
m4 = m3.drop('on which day the death occured',axis=1)
m5 = m4.drop('death',axis=0)
m6 = m5.drop('on which day the death occured',axis=0)


In [68]:
import scipy.spatial as sp
print(1- sp.distance.cdist(n6, m6, 'cosine').mean())

0.05623908732548977


In [55]:
def upper(df):
    '''Returns the upper triangle of a correlation matrix.
    You can use scipy.spatial.distance.squareform to recreate matrix from upper triangle.
    Args:
      df: pandas or numpy correlation matrix
    Returns:
      list of values from upper triangle
    '''
    try:
        assert(type(df)==np.ndarray)
    except:
        if type(df)==pd.DataFrame:
            df = df.values
        else:
            raise TypeError('Must be np.ndarray or pd.DataFrame')
    mask = np.triu_indices(df.shape[0], k=1)
    return df[mask]



In [56]:
# Now lets measure the similarity 
print(stats.spearmanr(upper(n6), upper(m6)))

SpearmanrResult(correlation=0.7843499903017942, pvalue=5.596006486999836e-198)
